In [243]:
from datetime import date
from prophet import Prophet
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import yfinance as yf

In [244]:
df = pd.read_csv("../Resources/btcjoin.csv", parse_dates=['date'])
btc_df = yf.download('BTC-USD')
btc_df = btc_df.reset_index()
btc_df = btc_df.loc[(btc_df['Date'] > '2022-10-25')]
btc_df['Close']=btc_df['Close'].astype("float")
df['price']=df['price'].str.replace(',','')
df['price']=df['price'].astype("float")
btc_df = btc_df.rename(columns={"Close": "price", "Date":"date"})
df = pd.merge(df, btc_df, on=['date', 'price'], how='outer')
df = df.rename(columns={"value": "wallets"})
df = df.drop(columns=['volume','change', 'low', 'high', 'open','Open','High','Low','Adj Close', 'Volume', 'Unnamed: 0', "wallets", "address", "mined"])
df['200D'] = df['price'].rolling(200).mean()
df['300D'] = df['price'].rolling(300).mean()
df['50D'] = df['price'].rolling(50).mean()
df = df.dropna()
df['meanavge'] = (df['200D'] + df['300D'] + df['50D'] )/3
df = df.drop(columns=['200D','300D', '50D'])
df['meanvalue'] = df["price"] - df["meanavge"]
df['status'] = df['meanvalue'].apply(lambda x: '1' if x > 0 else '0')
df['status']=df['status'].astype("object")
df['price-meanavge']=df['price'] - df['meanavge']
df['move%'] = df['price-meanavge']/(df['price'] + df['meanavge'])
bins = [-0.43, -0.18, 0, 0.18, 0.43]
group_names = ["Severely Oversold","Neutral Oversold", "Neutral Overbought","Severely Overbought"]
df["Valuation"] = pd.cut(df["move%"], bins, labels=group_names)
pricefrommean = df.meanvalue.iloc[-1].round(2)
currentzone = df.Valuation.iloc[-1]
delta1cycle = df.index[df['date']=='2015-09-15'].tolist()[0] - df.index[df['date']=='2013-12-04'].tolist()[0]
delta2cycle = df.index[df['date']=='2019-04-2'].tolist()[0] - df.index[df['date']=='2017-12-17'].tolist()[0] 
averageunder = int((delta1cycle+delta2cycle)/2)
delta1fromp2p = df.index[df['date']=='2017-03-17'].tolist()[0] - df.index[df['date']=='2013-12-04'].tolist()[0]
delta2fromp2p = df.index[df['date']=='2020-11-29'].tolist()[0] - df.index[df['date']=='2017-12-17'].tolist()[0] 
averagep2p = int((delta1fromp2p+delta2fromp2p)/2)
sincealltimehigh = df.index[-1] - df.index[df['price']==df.price.max()].tolist()[0]

[*********************100%***********************]  1 of 1 completed


In [245]:
df

,date,price,meanavge,meanvalue,status,price-meanavge,move%,Valuation
299,2011-07-21,13.600000,8.704611,4.895389,1,4.895389,0.219479,Severely Overbought
300,2011-07-22,13.700000,8.762722,4.937278,1,4.937278,0.219799,Severely Overbought
301,2011-07-23,13.700000,8.796167,4.903833,1,4.903833,0.217985,Severely Overbought
302,2011-07-24,14.000000,8.801778,5.198222,1,5.198222,0.227974,Severely Overbought
303,2011-07-25,14.100000,8.823000,5.277000,1,5.277000,0.230205,Severely Overbought
...,...,...,...,...,...,...,...,...
4465,2023-01-13,19909.574219,20325.315235,-415.741016,0,-415.741016,-0.010333,Neutral Oversold
4466,2023-01-14,20976.298828,20333.620223,642.678605,1,642.678605,0.015557,Neutral Overbought
4467,2023-01-15,20880.798828,20341.967059,538.831769,1,538.831769,0.013071,Neutral Overbought
4468,2023-01-16,21169.632812,20351.979523,817.653289,1,817.653289,0.019692,Neutral Overbought


In [246]:
import plotly.graph_objects as go

fig = go.Figure(go.Indicator(
    domain = {'x': [0, 1], 'y': [0, 1]},
    value = df.price.iloc[-1],
    mode = "gauge+number+delta",
    title = {'text': "Current Price and ATH Delta"},
    number = {'prefix': "$"},
    delta = {'reference': df.price[df['price']==df.price.max()].tolist()[0], "valueformat": ".0f"},
    gauge = {'axis': {'range': [None, df.price[df['price']==df.price.max()].tolist()[0]]},
    'bar': {'color': "orange"}}))

fig.show()
fig.write_html("../static/priceath.html")

In [247]:
import plotly.graph_objects as go


fig = go.Figure(go.Indicator(
    domain = {'x': [0, 1], 'y': [0, 1]},
    value = df.meanavge.iloc[-1],
    mode = "number+delta",
    number = {'prefix': "$"},
    delta = {"reference": ((df.meanavge.iloc[-2])), "valueformat": ".0f"},
    title = {'text': "Meanaverage Day Change"},
    gauge = {'axis': {'range': [None, df.price[df['price']==df.price.max()].tolist()[0]]},
    'bar': {'color': "orange"}}))


fig.show()
fig.write_html("../static/meanaverage.html")

In [248]:
import plotly.graph_objects as go

fig = go.Figure(go.Indicator(
    domain = {'x': [0, 1], 'y': [0, 1]},
    value =  df.price[df['price']==df.price.max()].tolist()[0],
    mode = "number",
    number = {'prefix': "$"},
    title = {'text': "Previous ATH"},
    gauge = {'axis': {'range': [None, df.price[df['price']==df.price.max()].tolist()[0]]},
    'bar': {'color': "orange"}}))

    

fig.show()
fig.write_html("../static/previousATH.html")

In [249]:
import plotly.graph_objects as go

fig = go.Figure(go.Indicator(
    domain = {'x': [0, 1], 'y': [0, 1]},
    value = df.price.iloc[-1] - df.meanavge.iloc[-1],
    mode = "number+delta",
    number = {'prefix': "$"},
    delta = {"reference": ((df.price.iloc[-2] - df.meanavge.iloc[-2])), "valueformat": ".0f"},
    title = {'text': "Current Price Meanaverage Delta Day Change"},
    gauge = {'axis': {'range': [None, df.price[df['price']==df.price.max()].tolist()[0]]},
    'bar': {'color': "orange"}}))

fig.show()
fig.write_html("../static/price-mean.html")

In [250]:
import plotly.graph_objects as go

fig = go.Figure(go.Indicator(
    mode = "gauge+number",
    title = {'text': "Current Valuation - Standard Deviation percent move from meanaverage", 'font': {'size': 15}},
    value = df['move%'].iloc[-1],
    domain = {'x': [0,1], 'y': [0,1]},
    gauge = {
        'axis': {'range': [-0.43, 0.43], 'tickwidth': 1, 'tickcolor': "darkblue"},
        'bar': {'color': "darkblue"},
        'bgcolor': "white",
        'borderwidth': 2,
        'bordercolor': "white",
        'steps': [
            {'range': [-0.43, -0.18], 'color': 'red'},
            {'range': [-0.18, 0], 'color': 'yellow'},
            {'range': [0, 0.18], 'color': 'blue'},
            {'range': [0.18,0.43], 'color': 'green'}],
        }))

fig.update_layout(
    font={'color': "black", 'family': "Arial"},
    xaxis={'showgrid': False, 'range':[-1,1]},
    yaxis={'showgrid': False, 'range':[0,1.11]},
    plot_bgcolor='rgba(0,0,0,0)'
    )

fig.add_annotation(
    ax=0,
    ay=0.4,
    axref='x',
    ayref='y',
    x=df['move%'].iloc[-1],
    y=0.9,
    xref='x',
    yref='y',
    showarrow=True,
    arrowhead=2,
    arrowsize=1,
    arrowwidth=4
    )

# Set the visibility ON
fig.update_yaxes(title='', visible=True, showticklabels=False)
# Set the visibility OFF
fig.update_xaxes(title='x', visible=False, showticklabels=False)

fig.show()
fig.write_html("../static/valuation.html")

In [251]:
bitcoin =yf.Ticker('BTC-USD').history(period='ytd',interval='1d',actions=False).reset_index()
bitcoin = ((((bitcoin.Close.iloc[-1]/bitcoin.Close.iloc[0])-1)))

spy500 = yf.Ticker('^GSPC').history(period='ytd',interval='1d',actions=False).reset_index()
spy500 = ((((spy500.Close.iloc[-1]/spy500.Close.iloc[0])-1)))

Gold = yf.Ticker('GC=F').history(period='ytd',interval='1d',actions=False).reset_index()
Gold = ((((Gold.Close.iloc[-1]/Gold.Close.iloc[0])-1)))

nas = yf.Ticker('^IXIC').history(period='ytd',interval='1d',actions=False).reset_index()
nas = ((((nas.Close.iloc[-1]/nas.Close.iloc[0])-1)))

DJ = yf.Ticker('^DJI').history(period='ytd',interval='1d',actions=False).reset_index()
DJ = ((((DJ.Close.iloc[-1]/DJ.Close.iloc[0])-1)))

Oil = yf.Ticker('Cl=F').history(period='ytd',interval='1d',actions=False).reset_index()
Oil = ((((Oil.Close.iloc[-1]/Oil.Close.iloc[0])-1)))

Rus = yf.Ticker('^RUT').history(period='ytd',interval='1d',actions=False).reset_index()
Rus = ((((Rus.Close.iloc[-1]/Rus.Close.iloc[0])-1)))

TLT = yf.Ticker('TLT').history(period='ytd',interval='1d',actions=False).reset_index()
TLT = ((((TLT.Close.iloc[-1]/TLT.Close.iloc[0])-1)))

DXY = yf.Ticker('DX=F').history(period='ytd',interval='1d',actions=False).reset_index()
DXY = ((((DXY.Close.iloc[-1]/DXY.Close.iloc[0])-1)))


data = [["Bitcoin", bitcoin], ["Gold", Gold], ["Oil", Oil],["20yr Treasury", TLT], ["DXY", DXY], ["Russell 2000 Index", Rus], ["S&P 500", spy500], ["NASDAQ", nas], ["Dow Jones", DJ]]

# Create the pandas DataFrame
data = pd.DataFrame(data, columns=['Asset', 'YTD%'])
data['YTD%'] = (data['YTD%'].astype(float) * 100).round(2)

In [252]:
import plotly.express as px
fig = px.bar(data, x='YTD%', y='Asset', orientation='h', text=str("YTD%"), color="YTD%",color_continuous_scale=px.colors.sequential.Oryel)
fig.layout.yaxis.tickformat = ',.%'
fig.update_layout(title_text='Year To Date Returns')
fig.update_layout(yaxis=dict(showticklabels=True))

fig.update_layout(
    margin=dict(l=4, r=100, t=70, b=20),
)

fig.write_html("../static/YTD.html")
fig.show()